[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/contextual_compression.ipynb)

# 文档检索中的上下文压缩

## 概述

此代码演示了使用LangChain和OpenAI语言模型在文档检索系统中实现上下文压缩。该技术旨在通过在给定查询的上下文中压缩和提取文档的最相关部分来提高检索信息的相关性和简洁性。

## 动机

传统的文档检索系统通常返回整个块或文档，这可能包含不相关的信息。上下文压缩通过智能地提取和压缩检索文档中最相关的部分来解决这个问题，从而实现更集中和高效的信息检索。

## 关键组件

1. 从PDF文档创建向量存储
2. 基础检索器设置
3. 基于LLM的上下文压缩器
4. 上下文压缩检索器
5. 集成压缩检索器的问答链

## 方法详情

### 文档预处理和向量存储创建

1. 使用自定义的`encode_pdf`函数处理PDF并将其编码到向量存储中。

### 检索器和压缩器设置

1. 从向量存储创建基础检索器。
2. 使用OpenAI的GPT-4模型初始化基于LLM的上下文压缩器（LLMChainExtractor）。

### 上下文压缩检索器

1. 将基础检索器和压缩器组合成ContextualCompressionRetriever。
2. 此检索器首先使用基础检索器获取文档，然后应用压缩器提取最相关的信息。

### 问答链

1. 创建一个RetrievalQA链，集成压缩检索器。
2. 此链使用压缩和提取的信息来生成查询的答案。

## 此方法的优点

1. 提高相关性：系统仅返回与查询最相关的信息。
2. 提高效率：通过压缩和提取相关部分，减少了LLM需要处理的文本量。
3. 增强上下文理解：基于LLM的压缩器可以理解查询的上下文并相应地提取信息。
4. 灵活性：系统可以轻松适应不同类型的文档和查询。

## 结论

文档检索中的上下文压缩提供了一种强大的方式来增强信息检索系统的质量和效率。通过智能地提取和压缩相关信息，它为查询提供更集中和上下文感知的响应。这种方法在需要从大型文档集合中进行高效和准确信息检索的各个领域都有潜在应用。

<div style="text-align: center;">

<img src="../images/contextual_compression.svg" alt="contextual compression" style="width:70%; height:auto;">
</div>

# 包安装和导入

下面的单元格安装运行此notebook所需的所有必要包。


In [ ]:
# 安装所需包
!pip install langchain python-dotenv

In [ ]:
# 克隆仓库以访问辅助函数和评估模块
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 如果需要使用最新数据运行
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA


# 为Colab兼容性替换原始路径追加
from helper_functions import *
from evaluation.evalute_rag import *

# 从.env文件加载环境变量
load_dotenv()

# 设置OpenAI API密钥环境变量
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### 定义文档路径

In [ ]:
# 下载所需数据文件
import os
os.makedirs('data', exist_ok=True)

# 下载此notebook中使用的PDF文档
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [2]:
path = "data/Understanding_Climate_Change.pdf"

### 创建向量存储

In [3]:
vector_store = encode_pdf(path)

### 创建检索器 + 上下文压缩器 + 组合它们

In [7]:
# 创建检索器
retriever = vector_store.as_retriever()


# 创建上下文压缩器
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
compressor = LLMChainExtractor.from_llm(llm)

# 将检索器与压缩器组合
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# 使用压缩检索器创建QA链
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True
)

### 使用示例

In [6]:
query = "文档的主要主题是什么？"
result = qa_chain.invoke({"query": query})
print(result["result"])
print("源文档:", result["source_documents"])

The main topic of the document is climate change, focusing on international collaboration, national strategies, policy development, and the ethical dimensions of climate justice. It discusses frameworks like the UNFCCC and the Paris Agreement, as well as the importance of sustainable practices for future generations.
Source documents: [Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 9}, page_content='Chapter 6: Global and Local Climate Action  \nInternational Collaboration  \nUnited Nations Framework Convention on Climate Change (UNFCCC)  \nThe UNFCCC is an international treaty aimed at addressing climate change. It provides a \nframework for negotiating specific protocols and agreements, such as the Kyoto Protocol and \nthe Paris Agreement. Global cooperation under the UNFCCC is crucial for coordinated \nclimate action.  \nParis Agreement  \nThe Paris Agreement, adopted in 2015, aims to limit global warming to well below 2 degrees \nCelsius above pre-i